# Klassifikation von Texten mithilfe von Transformern

*Transformer* sind seit ChatGPT in aller Munde. Mit den "kleinen Geschwistern" von GPT kann man sehr gut Texte klassifizieren und andere NLP-Aufgaben lösen.
Die folgende Beschreibung wie das funktioniert stammt übrigens zu großen Teilen von ChatGPT, lediglich an einigen Stellen habe ich etwas geändert (damit Sie auch noch selbst etwas zu tun haben).

Ja, ich kann Ihnen helfen. Um Twitter-Nachrichten mit einem Transformer-Modell zu klassifizieren, folgen Sie diesen Schritten:

1. Installieren Sie die erforderlichen Bibliotheken:


In [ ]:
! pip install torch transformers tqdm scikit-learn

2. Importieren Sie die erforderlichen Bibliotheken:


In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pandas as pd
import numpy as np
from tqdm import tqdm

3. Implementieren Sie eine benutzerdefinierte Dataset-Klasse

**Aufgabe 1: Ergänzen Sie Code zum Bereinigen der Tweets.**

In [ ]:
class GermEvalDataset(Dataset):
    def __init__(self, tokenizer, data_path, max_len):
        self.tokenizer = tokenizer
        self.data = pd.read_csv(data_path, sep='\t', header=None, names=['text', 'label', 'fine'])
        
        ### YOUR CODE HERE
        #   Clean Tweets
        ###
        
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = self.data.loc[index, 'text']
        label = self.data.loc[index, 'label']
        
        inputs = self.tokenizer.encode_plus(
            text,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_tensors='pt'
        )
        
        input_ids = inputs['input_ids'][0]
        attention_mask = inputs['attention_mask'][0]
        
        if label == "OTHER":
            label_tensor = torch.tensor(0)
        elif label == "OFFENSE":
            label_tensor = torch.tensor(1)
        else:
            raise ValueError(f"Invalid label: {label} for {text} at {index}")
            
        return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": label_tensor}

4. Laden Sie das pre-trained Modell und den Tokenizer:

**ChatGPT schlägt hier das Modell `"deepset/gbert-large"` vor – eine gute Wahl für deutschsprachige Tweets.
Recherchieren Sie im [Model-Hub von Higging Face](https://huggingface.co/models) ein paar Alternativen und vergleichen Sie die Ergebnisse.**

In [ ]:
model_name = "deepset/gbert-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

5. Erstellen Sie die DataLoader für Training und Validierung:

In [ ]:
MAX_LEN = 512
BATCH_SIZE = 16
train_data_path = "../data/GermEval-2018/germeval2018.training.txt"
val_data_path = "../data/GermEval-2018/germeval2018.test.txt"

train_dataset = GermEvalDataset(tokenizer, train_data_path, MAX_LEN)
val_dataset = GermEvalDataset(tokenizer, val_data_path, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

6. Erstellen Sie die Trainings-Argumente:

**Die Vorgaben von ChatGPT sind in Ordnung, aber schauen Sie einmal, was passiert, wenn Sie an den Parametern `BATCH_SIZE` und `learning_rate` "drehen".**

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=3,
    logging_steps=100,
    save_steps=1000,
    evaluation_strategy="steps",
    seed=42,
)

7. Definieren Sie die Metrik-Funktion:

**ChatGPT verwendet hier Metriken von *Scikit Learn*. Eine Alternative wären die Metriken aus `huggingface.co/metrics`.**

In [ ]:
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    targets = p.label_ids
    precision = precision_score(targets, preds)
    recall = recall_score(targets, preds)
    f1 = f1_score(targets, preds)
    acc = accuracy_score(targets, preds)
    return {"precision": precision, "recall": recall, "f1": f1, "accuracy": acc}

8. Erstellen Sie einen Trainer und trainieren Sie das Modell:


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

9. Optional: Bewerten Sie das Modell nach dem Training:

```python
trainer.evaluate()
```

Das trainierte Modell kann jetzt zur Klassifikation von Twitter-Nachrichten verwendet werden.